## Importing Libraries

In [1]:
!pip install -q keras-tcn --no-dependencies
from tcn import TCN, tcn_full_summary

In [2]:
import json
import numpy as np
from termcolor import colored
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import tokenizer_from_json

In [3]:
maxlength = 219
domains = ['Sports', 'Business & Economics', 'Science & Technology', 'Wellness', 'Politics', 'Travel', 'Entertainment']

## Load the tokenizer from the JSON file

In [4]:
with open('Trained Models for 07 Classes/Tokenizer Used During Training/7_classes_tokenizer.json', 'r', encoding='utf-8') as json_file:
    tokenizer_json = json.load(json_file)
    tokenizer = tokenizer_from_json(tokenizer_json)

## Texts to Predict

In [5]:
sinhala_texts = [
                 'ඉන්දීය කණ්ඩායම මෙවර ලෝක කුසලාන ක්‍රිකට් තරගාවලියේ අවසන් පූර්ව වටයට අද සුදුසුකම් ලබා ගත්තේ ය. ඒ, ශ්‍රී ලංකාවට එරෙහිව මුම්බායිහිදී පැවැති තරගය ලකුණු 302කින් ජය ගැනීමත් සමගයි. තරගයේ කාසියේ වාසිය දිනූ ශ්‍රී ලංකා නායක කුසල් මෙන්ඩිස් පළමුවෙන් පන්දු රැකීමට තීරණය කළා. ඒ අනුව පළමුවෙන් පන්දුවට පහර දීමට පිටියට පිවිසි ඉන්දීය කණ්ඩායමේ නායක රෝහිත් ශර්මා පිටියෙන් ඉවත් වුණේ ලකුණු 4කටයි. අනතුරුව විරාත් කෝලි සමග එක් වූ ශුබ්මන් ගිල් ඉන්දීය ඉනිම ගොඩනැගුවේය. ඒ, දෙවැනි කඩුල්ලට ලකුණු 189ක සබදතාවයක් එක් කරමින්. ශුබ්මන් ගිල් ලකුණු 92ක් රැස් කළ අතර, ඔහුගේ ඉනිමට 4 පහර 11ක් සහ 6 පහර 2ක් ඇතුළත් විය. 49 වැනි එක් දින ශතකය අබියස සිටි විරාත් කෝලි ලකුණු 88කට දැවී ගියේ ඩිල්ශාන් මදුශංකගේ පන්දුවක් හමුවේයි. ලෝකේෂ් රාහුල්  සහ ශ්‍රෙයාස් අයියර් සිව්වැනි කඩුල්ලට ලකුණු 60ක සබඳතාවක් ගොඩනැගුවේය. ශ්‍රෙයාස් අයියර් 4 පහර 3ක් සහ 6 පහර 6ක්ද සමගින් ලකුණු 82යි. ඩිල්ෂාන් මදුශංක කඩුලු 5ක් දවා ගත්තා. ඉන්දීය කණ්ඩායම නියමිත පන්දුවාර 50 අවසානයේදී කඩුලු 8කට ලකුණු 357ක් රැස් කළේ ය. ලකුණු 357ක ඉලක්කයකින් පිටියට පිවිසි ශ්‍රී ලංකා කණ්ඩායමේ කිසිදු පිතිකරුවෙකුට ලකුනු අතරට පිවිසීමට ඉන්දීය පන්දු යවන්නන් අවස්ථාව ලබා දුන්නේ නැහැ. ඒ අනුව ලකුණු 3කදී ශ්‍රී ලංකා මුල් පෙළ කඩුලු 4 බිඳ වැටුනා. පැතුම් නිස්සංක, දිමුත් කරුණාරත්න සහ සදීර සමරවික්‍රමට ලකුණු රැස් කිරීමට හැකි වුණේ නැහැ. නායක කුසල් මෙන්ඩිස් එක් ලකුණකට දැවී ගියා. ලකුණු 29කදී ශ්‍රී ලංකා කණ්ඩායාමේ කඩුලු 8ක් බිඳ වැටුණි. එහිදී පිතිකරුවන් පස් දෙනෙකුම ලකුණු රැස් කිරීමට පෙර දවා ගැනීමට ඉන්දීය පන්දු යවන්නන් සමත් වුණා. ඇන්ජලෝ මැතිව්ස් ලකුණු 12යි. ඉනිමේ වැඩිම ලකුණු ලාභියා වූ කසුන් රාජිත ලකුණු 14යි. මොහොමඩ් ශාමි ලකුණු 18කට කඩුලු 8ක් දවා ගත්තේ ය. ඒ අනුව ලෝක කුසලාන ක්‍රිකට් තරගාවලි අංශයෙන් වැඩිම කඩුලු සංඛ්‍යාවක් දවා ගත් ඉන්දීය පන්දු යවන්නා ලෙස ඔහු වාර්තා අතරට එක් වුණේ කඩුලු 45ක් දවා ගනිමින් ය. පන්දුවාර 19යි පන්දු 4කදී ශ්‍රී ලංකා කණ්ඩායමේ සියලු දෙනා ලකුණු 55කට දැවී ගියේ ය. ඒ අනුව ඉන්දීය කණ්ඩායම අද වාර්තා අතරට එක් වුණේ, ලෝක කුසලාන ක්‍රිකට් තරගයක දෙවැනියට වැඩිම ලකුණු පරතරයකින් තරගයක් ජය ගත් කණ්ඩායම ලෙසින් ය.', # Sports
                 'ආනයනික සීනි කිලෝග්‍රෑමයක් සඳහා පනවා තිබූ විශේෂ වෙළඳ භාණ්ඩ බද්ද ඉහළ දමා තිබේ. මුදල් අමාත්‍යංශය සඳහන් කළේ,  ශත 25 ක්ව පැවති එම බද්ද රුපියල් 50 දක්වා ඉහළ දමා ඇති බවයි. ඒ , අද සිට ක්‍රියාත්මක වන පරිදියි. අද සිට වසරක් සඳහා මෙය ක්‍රියාත්මක බව මුදල් අමාත්‍යාංශය සඳහන් කළේ ය. ', # Business & Economics
                 'ඒ අනුව Whatsapp පරිශීලකයන්ට තමන් තොරතුරු හුවමාරු කරගන්නා "conversations" වෙන අයෙකුට බලාගත නොහැකි ආකාරයට සඟවා තැබීමට හැකියාව ලැබේ. යම්හෙයකින් අදාල conversations එක හා සම්බන්ධවීමට රහස් මුරපදය හෝ ඇඟිලි සලකුණ යෙදීමට සිදුවෙයි. Whatsapp යෙදුමේ අයිතිය සතු Meta ආයතනය පවසන්නේ පරිශීලකයන්ගේ රහස්‍යභාවය ආරක්ෂා කිරීමේ අරමුණ ඇතිව මෙම නව පියවර ගත් බවයි. ඉදිරි සති කිහිපය තුළ ChatLock සඳහා තවත් නවතම ආරක්ෂිත අංග එක් කිරීමට කටයුතු කරන බව Meta සමාගම නිවේදනය කරයි. ඉදිරියේ දී ඔබේ පුද්ගලික සංවාද ද මෙලෙස සඟවා තබා ගැනීමට හැකියාව ලැබෙනු ඇති.', # Science & Technology
                 'ව්‍යායාම කිරීම අපට හොඳ බව අපි කවුරුත් දනිමු, නමුත් එහි ප්‍රතිලාභ සැමවිටම අනතුරු ඇඟවීමක් කිරීමට සහ අපගේ ධාවන සපත්තු ලේස් කිරීමට අපව පොලඹවන්නේ නැත. එක්සත් ජනපදයේ සෞඛ්‍ය සංඛ්‍යාලේඛන සඳහා වන ජාතික මධ්‍යස්ථානයට අනුව, ඇමරිකානුවන්ගෙන් සියයට 75 ක් වායුගෝලීය සහ ශක්තිමත් කිරීමේ ව්‍යායාම සඳහා නිර්දේශිත මාර්ගෝපදේශ සපුරාලන්නේ නැත. බොහෝ ප්‍රවීණයන් පවසන්නේ වඩා හොඳ සහ නිතිපතා ව්‍යායාම සඳහා යතුර ශරීරයේ නොව මනසෙහි බවයි. Netflix බින්ජ් එකක් සහ සන්ධ්‍යා ධාවනයක් අතර තීරණය කරන ඕනෑම කෙනෙකුට තේරුම් ගත හැකි පරිදි, ශරීරය කැමැත්තෙන් සිටිය හැක, නමුත් ආත්මයට ඉඳහිට කික්-ආරම්භයක් අවශ්‍ය වේ.', # Wellness
                 'බ්‍රිතාන්‍ය පාර්ලිමේන්තු සම්ප්‍රදාය අනුව සකස් කරන ලද පාර්ලිමේන්තු ප්‍රමිති පිළිබද පනත් කෙටුම්පතට කැබිනට් අනුමැතිය හිමිවී තිබේ. හිටපු කතානායක කරූ ජයසූරිය මහතා සඳහන් කළේ, අග්‍රාමාත්‍යවරයා සහ අධිකරණ අමාත්‍යවරයා පක්ෂ නායකයින් සමග මේ පිළිබදව සාකච්ඡා කරන බවයි. එම පනත් කෙටුම්පත සකස් කර ඇත්තේ, පාර්ලිමේන්තු මන්ත්‍රීවරුන්ගේ විනය සම්බන්ධයෙන් රෙගුලාසි ඇතුළත් කරමිනි. හිටපු කතානායකවරයා සඳහන් කළේ, පාර්ශව රැසක අදහස් විමසමින් මෙරටට ගැළපෙන පරිදි අදාළ කෙටුම්පත සකස් කර ඇති බවයි. පාර්ලිමේන්තුවේ සියලු කටයුතු ආවරණය වන පරිදි එම කෙටුම්පත සකස් කර ඇත. කතානායක කරූ ජයසුරිය මහතා මෙම පනත් කෙටුම්පත සකස් කිරීමට පත් කළ කමිටුවේ සාමාජිකයෙකි.', # Politics
                 'Travel Triangle නම් සංචාරක වෙබ් අඩවියේ නවතම වාර්තාවන්ට අනුව මෙලෙස ශ්‍රී ලංකාව සංචාරය සඳහා සුදුසු පළමු රට බවට පත්ව ඇත්තේ මෙරට ස්වභාවික සම්පත්වල සුන්දරත්වය හේතුවෙනි. රන්වන් වෙරළ, වන සතුගෙන් පොහොසත් වනාන්තර පද්ධතිය, මනස්කාන්ත තේ වතු සහ මීදුමෙන් වැසී ගිය කදු පන්ති සහ අනෙකුත් සංචාරක ආකර්ෂණය දිනා ගන්නා ස්ථාන පිළිබඳව මෙම වෙබ් අඩවිය කරුණු දක්වා තිබේ. අදාළ වෙබ් අඩවිය සඳහන් කරන්නේ, මෙම දූපතේ සුන්දරත්වය හා භෞතික විවිධත්වය ගැන සංචාරකයින් මවිත වන බව ය. ශ්‍රී ලංකාවේ සිව්වන උසම කදු මුදුන වන ශ්‍රී පාද කදු මුදුණේ සිට දකින දර්ශනය සිත් ඇදගන්නා සුළු එකක් බවත්, ඔබේ ශරීර යෝග්‍යතාවයට අනුව පැය 5-6 ඇතුළත ඔබටම මෙම ලෝක උරුම අඩයට පියනැගිය හැකි බවත් Travel Triangle වෙබ් අඩවිය සඳහන් කර තිබේ. ඔබේ ජීවිතය යළි පණ ගන්වා ගැනීමට අවශ්‍යනම්, නැගෙනහිර වෙරළ තීරයේ සෙවන සහිත ස්ථානයක අසුන් ගෙන විටමින් ඩී ලබා ගන්න යනුවෙන් ද ඔවුන් ශ්‍රී ලංකාව පිළිබඳව සටහන් තබා තිබේ.', # Travel
                 'මෙවර සම්මාන උළෙලේදී වසර 62කට පසු රතු පලස භාවිත නොවීම විශේෂත්වයකි. ඔස්කාර් සංවිධායකයින් පෙන්වා දෙන්නේ , මෙවර ෂැම්පේන් වර්ණයෙන් යුත් පලසක් භාවිත කරන බවයි. මෙවර සම්මාන උළෙලේ හොඳම සහය නළුවාට හිමි සම්මානය හිමි වුණේ, එවිරිතින්ග් එවිරිවෙයා ඕල් ඇටි වන්ස් (Everything Everywhere All at Once ) හි රංගන දායකත්වය වෙනුවෙන් කී වී ක්වාන්ටයි Ke Huy Quan ය. එම චිත්‍රපටයේම රංගනය වෙනුවෙන් ජේමී ලී කාර්ටස් හොඳම සහය නිළියට හිමි සම්මානය දිනා ගත්තේ ය. එම සම්මානය සඳහා ඇය නිර්දේශ වූ පළමු අවස්ථාවේදීම සම්මානය හිමි වීම විශේෂත්වයකි.' # Entertainment
                ]

In [6]:
# sinhala_texts = [
#                  'ඉන්දීය කණ්ඩායම මෙවර ලෝක කුසලාන ක්‍රිකට් තරගාවලියේ අවසන් පූර්ව වටයට අද සුදුසුකම් ලබා ගත්තේ ය. ඒ, ශ්‍රී ලංකාවට එරෙහිව මුම්බායිහිදී පැවැති තරගය ලකුණු 302කින් ජය ගැනීමත් සමගයි. තරගයේ කාසියේ වාසිය දිනූ ශ්‍රී ලංකා නායක කුසල් මෙන්ඩිස් පළමුවෙන් පන්දු රැකීමට තීරණය කළා. ඒ අනුව පළමුවෙන් පන්දුවට පහර දීමට පිටියට පිවිසි ඉන්දීය කණ්ඩායමේ නායක රෝහිත් ශර්මා පිටියෙන් ඉවත් වුණේ ලකුණු 4කටයි. අනතුරුව විරාත් කෝලි සමග එක් වූ ශුබ්මන් ගිල් ඉන්දීය ඉනිම ගොඩනැගුවේය. ඒ, දෙවැනි කඩුල්ලට ලකුණු 189ක සබදතාවයක් එක් කරමින්. ශුබ්මන් ගිල් ලකුණු 92ක් රැස් කළ අතර, ඔහුගේ ඉනිමට 4 පහර 11ක් සහ 6 පහර 2ක් ඇතුළත් විය. 49 වැනි එක් දින ශතකය අබියස සිටි විරාත් කෝලි ලකුණු 88කට දැවී ගියේ ඩිල්ශාන් මදුශංකගේ පන්දුවක් හමුවේයි. ලෝකේෂ් රාහුල්  සහ ශ්‍රෙයාස් අයියර් සිව්වැනි කඩුල්ලට ලකුණු 60ක සබඳතාවක් ගොඩනැගුවේය. ශ්‍රෙයාස් අයියර් 4 පහර 3ක් සහ 6 පහර 6ක්ද සමගින් ලකුණු 82යි. ඩිල්ෂාන් මදුශංක කඩුලු 5ක් දවා ගත්තා. ඉන්දීය කණ්ඩායම නියමිත පන්දුවාර 50 අවසානයේදී කඩුලු 8කට ලකුණු 357ක් රැස් කළේ ය. ලකුණු 357ක ඉලක්කයකින් පිටියට පිවිසි ශ්‍රී ලංකා කණ්ඩායමේ කිසිදු පිතිකරුවෙකුට ලකුනු අතරට පිවිසීමට ඉන්දීය පන්දු යවන්නන් අවස්ථාව ලබා දුන්නේ නැහැ. ඒ අනුව ලකුණු 3කදී ශ්‍රී ලංකා මුල් පෙළ කඩුලු 4 බිඳ වැටුනා. පැතුම් නිස්සංක, දිමුත් කරුණාරත්න සහ සදීර සමරවික්‍රමට ලකුණු රැස් කිරීමට හැකි වුණේ නැහැ. නායක කුසල් මෙන්ඩිස් එක් ලකුණකට දැවී ගියා. ලකුණු 29කදී ශ්‍රී ලංකා කණ්ඩායාමේ කඩුලු 8ක් බිඳ වැටුණි. එහිදී පිතිකරුවන් පස් දෙනෙකුම ලකුණු රැස් කිරීමට පෙර දවා ගැනීමට ඉන්දීය පන්දු යවන්නන් සමත් වුණා. ඇන්ජලෝ මැතිව්ස් ලකුණු 12යි. ඉනිමේ වැඩිම ලකුණු ලාභියා වූ කසුන් රාජිත ලකුණු 14යි. මොහොමඩ් ශාමි ලකුණු 18කට කඩුලු 8ක් දවා ගත්තේ ය. ඒ අනුව ලෝක කුසලාන ක්‍රිකට් තරගාවලි අංශයෙන් වැඩිම කඩුලු සංඛ්‍යාවක් දවා ගත් ඉන්දීය පන්දු යවන්නා ලෙස ඔහු වාර්තා අතරට එක් වුණේ කඩුලු 45ක් දවා ගනිමින් ය. පන්දුවාර 19යි පන්දු 4කදී ශ්‍රී ලංකා කණ්ඩායමේ සියලු දෙනා ලකුණු 55කට දැවී ගියේ ය. ඒ අනුව ඉන්දීය කණ්ඩායම අද වාර්තා අතරට එක් වුණේ, ලෝක කුසලාන ක්‍රිකට් තරගයක දෙවැනියට වැඩිම ලකුණු පරතරයකින් තරගයක් ජය ගත් කණ්ඩායම ලෙසින් ය.', # Sports
#                 ]

## Ensemble TCN-BiLSTM Model Prediction

In [7]:
ensemble_tcnbilstm_model = load_model('Trained Models for 07 Classes/Ensemble TCN-BiLSTM/7_classes_ensemble_tcnbilstm_best_model.h5', custom_objects={'TCN': TCN})

In [8]:
def modelDemo(sinhala_texts, tokenizer):
    txt_sequence = pad_sequences(tokenizer.texts_to_sequences(sinhala_texts), maxlen=maxlength)
    txt_prediction = [domains[np.argmax(i)] for i in ensemble_tcnbilstm_model.predict(txt_sequence)]
    
    for sin_txt, domain in zip(sinhala_texts, txt_prediction):
        print('{} - {}'.format(colored(sin_txt, 'yellow'), colored(domain, 'blue')))
    
modelDemo(sinhala_texts, tokenizer)

1/1 [==============================] - 3s 3s/step
ඉන්දීය කණ්ඩායම මෙවර ලෝක කුසලාන ක්‍රිකට් තරගාවලියේ අවසන් පූර්ව වටයට අද සුදුසුකම් ලබා ගත්තේ ය. ඒ, ශ්‍රී ලංකාවට එරෙහිව මුම්බායිහිදී පැවැති තරගය ලකුණු 302කින් ජය ගැනීමත් සමගයි. තරගයේ කාසියේ වාසිය දිනූ ශ්‍රී ලංකා නායක කුසල් මෙන්ඩිස් පළමුවෙන් පන්දු රැකීමට තීරණය කළා. ඒ අනුව පළමුවෙන් පන්දුවට පහර දීමට පිටියට පිවිසි ඉන්දීය කණ්ඩායමේ නායක රෝහිත් ශර්මා පිටියෙන් ඉවත් වුණේ ලකුණු 4කටයි. අනතුරුව විරාත් කෝලි සමග එක් වූ ශුබ්මන් ගිල් ඉන්දීය ඉනිම ගොඩනැගුවේය. ඒ, දෙවැනි කඩුල්ලට ලකුණු 189ක සබදතාවයක් එක් කරමින්. ශුබ්මන් ගිල් ලකුණු 92ක් රැස් කළ අතර, ඔහුගේ ඉනිමට 4 පහර 11ක් සහ 6 පහර 2ක් ඇතුළත් විය. 49 වැනි එක් දින ශතකය අබියස සිටි විරාත් කෝලි ලකුණු 88කට දැවී ගියේ ඩිල්ශාන් මදුශංකගේ පන්දුවක් හමුවේයි. ලෝකේෂ් රාහුල්  සහ ශ්‍රෙයාස් අයියර් සිව්වැනි කඩුල්ලට ලකුණු 60ක සබඳතාවක් ගොඩනැගුවේය. ශ්‍රෙයාස් අයියර් 4 පහර 3ක් සහ 6 පහර 6ක්ද සමගින් ලකුණු 82යි. ඩිල්ෂාන් මදුශංක කඩුලු 5ක් දවා ගත්තා. ඉන්දීය කණ්ඩායම නියමිත පන්දුවාර 50 අවසානයේදී කඩුලු 8කට ලකුණු 357ක් රැස් කළේ ය. ලකුණු 357ක ඉල